In [112]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set a few plotting defaults
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 18
plt.rcParams['patch.edgecolor'] = 'k'

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import warnings

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import *

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings(action='ignore')
sns.set_palette('pastel')

from lightgbm import LGBMClassifier
from lightgbm import plot_importance
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt

import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42)
import optuna
import lightgbm as lgb
from lightgbm import LGBMRegressor
import joblib
from sklearn.multioutput import RegressorChain
from numpy.linalg import norm
from numpy import dot
from sklearn.preprocessing import MinMaxScaler

In [4]:
def lg_nrmse(gt, preds):
    
    gt = pd.DataFrame(gt)
    preds = pd.DataFrame(preds)

    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = mean_squared_error(gt.iloc[:,idx], preds.iloc[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt.iloc[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:14])
    return score

In [6]:
kf = KFold(n_splits = 5)

In [8]:
load_study = joblib.load("optuna_tuning_model_xgb2.pkl")
print("Best trial until now:")
print(" Value: ", load_study.best_trial.value)
print(" Params: ")
for key, value in load_study.best_trial.params.items():
    print(f"    {key}: {value}")

Best trial until now:
 Value:  0.3064035354045282
 Params: 
    learning_rate: 0.005523604453900754
    n_estimators: 1900
    colsample_bylevel: 0.958300222028654
    colsample_bynode: 0.9458322893062406
    reg_lambda: 0.02295808647856259
    reg_alpha: 0.01550849621161686
    subsample: 0.6
    min_child_weight: 10
    gamma: 0.38614221022401557


# 1. 데이터 셋 준비

1) 기본 데이터셋(X_03/07)

In [254]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [255]:
train.drop(['X_04','X_23','X_47','X_48','X_10','X_11','ID'], axis = 1, inplace= True)
test.drop(['X_04','X_23','X_47','X_48','X_10','X_11','ID'], axis = 1, inplace= True)

In [256]:
train['X_3/7'] = train['X_03']/train['X_07']
test['X_3/7'] = test['X_03']/test['X_07']

2) Factor Analysis 변수 추가

In [10]:
train_fa=pd.read_csv('train_fa.csv')
test_fa=pd.read_csv('test_fa.csv')
train_fa=pd.concat([train,train_fa],axis=1)
test_fa=pd.concat([test,test_fa],axis=1)

3) 커넥터 핀 치수 편차 변수 추가

In [260]:
train_co=train.copy()
test_co=test.copy()
train_con = train[['X_24','X_25','X_26','X_27','X_28','X_29']]
test_con = test[['X_24','X_25','X_26','X_27','X_28','X_29']]
train_co['X_con'] = train_con.std(axis=1)
test_co['X_con'] = test_con.std(axis=1)

4) 레이돔 치수 편차 변수 추가

In [261]:
train_ra=train.copy()
test_ra=test.copy()
train_rad = train[['X_41','X_42','X_43','X_44']]
test_rad = test[['X_41','X_42','X_43','X_44']]
train_ra['X_rad'] = train_rad.std(axis=1)
test_ra['X_rad'] = test_rad.std(axis=1)

5) 안테나 패드 위치 편차

In [262]:
train_an=train.copy()
test_an=test.copy()
train_ant = train[['X_14','X_15','X_16','X_17','X_18']]
test_ant = test[['X_14','X_15','X_16','X_17','X_18']]
train_an['X_ant'] = train_ant.std(axis=1)
test_an['X_ant'] = test_ant.std(axis=1)

6) 스크류 체결 속도

In [263]:
train_sc=train.copy()
test_sc=test.copy()
train_sc['X_T1'] = train_sc['X_19']/train_sc['X_34']
train_sc['X_T2'] = train_sc['X_20']/train_sc['X_35']
train_sc['X_T3'] = train_sc['X_21']/train_sc['X_36']
train_sc['X_T4'] = train_sc['X_22']/train_sc['X_37']
test_sc['X_T1'] = test_sc['X_19']/test_sc['X_34']
test_sc['X_T2'] = test_sc['X_20']/test_sc['X_35']
test_sc['X_T3'] = test_sc['X_21']/test_sc['X_36']
test_sc['X_T4'] = test_sc['X_22']/test_sc['X_37']

In [29]:
train_x = train.filter(regex='X') 
train_y = train.filter(regex='Y') 
train_x_fa = train_fa.filter(regex='X')
train_x_co = train_co.filter(regex='X') 
train_x_ra = train_ra.filter(regex='X') 
train_x_an = train_an.filter(regex='X') 
train_x_sc = train_sc.filter(regex='X') 

# 2. 모델 적합 및 Predict 준비

In [28]:
xg = XGBRegressor(**load_study.best_trial.params, tree_method='gpu_hist', gpu_id=0,colsample_bytree= 0.8988148571795371,max_depth=9)
model1 = MultiOutputRegressor(estimator=xg)
model2 = MultiOutputRegressor(estimator=xg)
model3 = MultiOutputRegressor(estimator=xg)
model4 = MultiOutputRegressor(estimator=xg)
model5 = MultiOutputRegressor(estimator=xg)
model6 = MultiOutputRegressor(estimator=xg)

In [32]:
model1.fit(train_x, train_y)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=0.958300222028654,
                                            colsample_bynode=0.9458322893062406,
                                            colsample_bytree=0.8988148571795371,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            gamma=0.38614221022401557, gpu_id=0,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.005523604453900754,
                                            max_bin=Non

In [33]:
model2.fit(train_x_fa, train_y)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=0.958300222028654,
                                            colsample_bynode=0.9458322893062406,
                                            colsample_bytree=0.8988148571795371,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            gamma=0.38614221022401557, gpu_id=0,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.005523604453900754,
                                            max_bin=Non

In [34]:
model3.fit(train_x_co, train_y)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=0.958300222028654,
                                            colsample_bynode=0.9458322893062406,
                                            colsample_bytree=0.8988148571795371,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            gamma=0.38614221022401557, gpu_id=0,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.005523604453900754,
                                            max_bin=Non

In [35]:
model4.fit(train_x_ra, train_y)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=0.958300222028654,
                                            colsample_bynode=0.9458322893062406,
                                            colsample_bytree=0.8988148571795371,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            gamma=0.38614221022401557, gpu_id=0,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.005523604453900754,
                                            max_bin=Non

In [36]:
model5.fit(train_x_an, train_y)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=0.958300222028654,
                                            colsample_bynode=0.9458322893062406,
                                            colsample_bytree=0.8988148571795371,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            gamma=0.38614221022401557, gpu_id=0,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.005523604453900754,
                                            max_bin=Non

In [37]:
model6.fit(train_x_sc, train_y)

MultiOutputRegressor(estimator=XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=0.958300222028654,
                                            colsample_bynode=0.9458322893062406,
                                            colsample_bytree=0.8988148571795371,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            gamma=0.38614221022401557, gpu_id=0,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=0.005523604453900754,
                                            max_bin=Non

In [38]:
# joblib.dump(model1, "Attention_1.pkl")
# joblib.dump(model2, "Attention_2.pkl")
# joblib.dump(model3, "Attention_3.pkl")
# joblib.dump(model4, "Attention_4.pkl")
# joblib.dump(model5, "Attention_5.pkl")
# joblib.dump(model6, "Attention_6.pkl")

['Attention_6.pkl']

In [42]:
train_pred1=model1.predict(train_x)
train_pred2=model2.predict(train_x_fa)
train_pred3=model3.predict(train_x_co)
train_pred4=model4.predict(train_x_ra)
train_pred5=model5.predict(train_x_an)
train_pred6=model6.predict(train_x_sc)

In [264]:
test_pred1=model1.predict(test)
test_pred2=model2.predict(test_fa)
test_pred3=model3.predict(test_co)
test_pred4=model4.predict(test_ra)
test_pred5=model5.predict(test_an)
test_pred6=model6.predict(test_sc)

In [124]:
for i, pred in enumerate([train_pred1,train_pred2,train_pred3,train_pred4,train_pred5,train_pred6]):
    print(lg_nrmse(train_y, pred))

1.4607618904029713
1.437476605383265
1.4541822592489932
1.45647486069629
1.4558137311392225
1.4516914086404993


# 3.Ensemble with Attention

In [79]:
def softmax(a) :
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y=exp_a / sum_exp_a
    
    return y

In [362]:
def attention_score_matrix(target,dfs):
    target_arr=np.array(target)
    attention_score=np.empty(shape=(len(dfs),target.shape[1]))
    model_name=[]
    for i, pred in enumerate(dfs):
        attention_score[i,:]=np.diag(target_arr.T.dot(pred))
        model_name.append("model{}".format(i+1))
    as_matrix=pd.DataFrame(attention_score, columns =target.columns, index =model_name)
    
    return as_matrix

In [363]:
def attention_distribution_matrix(as_matrix):
    scaler=MinMaxScaler()
    scaler_result = scaler.fit_transform(as_matrix)
    minmax_as_matrix=pd.DataFrame(scaler_result,columns =as_matrix.columns, index =as_matrix.index)
    ad_matrix=pd.DataFrame()
    for i in range(1,as_matrix.shape[1]+1):
        if i<10:
            ad_matrix['Y_0{}'.format(i)]=softmax(np.array(minmax_as_matrix['Y_0{}'.format(i)]))
        else:
            ad_matrix['Y_{}'.format(i)]=softmax(np.array(minmax_as_matrix['Y_{}'.format(i)]))
    ad_matrix.index=as_matrix.index
    
    return ad_matrix

In [367]:
test_preds=[test_pred1,test_pred2,test_pred3,test_pred4,test_pred5,test_pred6]

In [364]:
train_preds=[train_pred1,train_pred2,train_pred3,train_pred4,train_pred5,train_pred6]
as_matrix=attention_score_matrix(train_y,train_preds)
ad_matrix=attention_distribution_matrix(as_matrix)
display(as_matrix)
display(ad_matrix)

,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
model1,73954.452620,45973.515770,42090.114906,7.442598e+06,3.885604e+07,1.090416e+07,396329.445384,2.738983e+07,2.741829e+07,1.988588e+07,2.344305e+07,2.727093e+07,2.726274e+07,2.728781e+07
model2,74014.261802,46050.975409,42162.574712,7.445575e+06,3.885940e+07,1.090444e+07,396409.456306,2.739008e+07,2.741848e+07,1.988614e+07,2.344337e+07,2.727115e+07,2.726299e+07,2.728793e+07
model3,73971.648309,45987.549547,42109.167231,7.443770e+06,3.885696e+07,1.090443e+07,396339.349777,2.738985e+07,2.741833e+07,1.988602e+07,2.344317e+07,2.727099e+07,2.726278e+07,2.728783e+07
model4,73956.141033,45988.010712,42096.543749,7.443661e+06,3.885718e+07,1.090475e+07,396343.019907,2.738988e+07,2.741831e+07,1.988589e+07,2.344310e+07,2.727097e+07,2.726280e+07,2.728781e+07
model5,73971.309676,45987.159144,42105.553118,7.443425e+06,3.885666e+07,1.090454e+07,396334.122224,2.738985e+07,2.741829e+07,1.988593e+07,2.344312e+07,2.727096e+07,2.726285e+07,2.728780e+07
model6,73969.806825,46005.167746,42113.513816,7.444280e+06,3.885749e+07,1.090550e+07,396352.707182,2.738990e+07,2.741836e+07,1.988599e+07,2.344317e+07,2.727100e+07,2.726282e+07,2.728785e+07


,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
model1,0.114965,0.113378,0.114542,0.102992,0.108876,0.110218,0.118588,0.118404,0.120074,0.108004,0.110568,0.115393,0.109095,0.123373
model2,0.312508,0.308194,0.311358,0.279962,0.295956,0.136376,0.322355,0.321855,0.316340,0.293587,0.300556,0.313670,0.296550,0.314924
model3,0.153261,0.135898,0.148990,0.152681,0.143377,0.135266,0.134215,0.130107,0.145506,0.186459,0.161765,0.147709,0.129690,0.144574
model4,0.118257,0.136709,0.125169,0.147186,0.153091,0.171699,0.140515,0.144568,0.129301,0.113508,0.127356,0.139284,0.142092,0.122820
model5,0.152395,0.135215,0.141741,0.135959,0.131132,0.146835,0.125726,0.130049,0.116375,0.133959,0.138659,0.129693,0.169778,0.115854
model6,0.148614,0.170606,0.158201,0.181221,0.167568,0.299605,0.158600,0.155017,0.172404,0.164483,0.161096,0.154251,0.152794,0.178455


In [371]:
def attention_predict(ad_matrix,dfs,target=None):
    for i in range(1,ad_matrix.shape[1]+1):
        for j, pred in enumerate(dfs):
            if j==0:
                temp_y_pred_total=np.vstack(pred[:,i-1]).T
            else:
                temp_y_pred=np.vstack(pred[:,i-1])
                temp_y_pred_total=np.vstack([temp_y_pred_total,temp_y_pred.T])
        locals()['df_y{}_pred'.format(i)]=temp_y_pred_total
    
    if type(target)==type(None):
        test_pred_at_arr=np.empty(shape=(dfs[0].shape[0],dfs[0].shape[1]))
        for i in range(1,ad_matrix.shape[1]+1):
            test_pred_at_arr[:,i-1]=locals()['df_y{}_pred'.format(i)].T.dot(np.array(ad_matrix)[:,i-1])
        
        return test_pred_at_arr
    else:
        train_pred_at_arr=np.empty(shape=(dfs[0].shape[0],dfs[0].shape[1]))
        for i in range(1,ad_matrix.shape[1]+1):
            train_pred_at_arr[:,i-1]=locals()['df_y{}_pred'.format(i)].T.dot(np.array(ad_matrix)[:,i-1])
        train_pred_at=pd.DataFrame(train_pred_at_arr,index=target.index,columns=target.columns) 
        print(lg_nrmse(train_y, train_pred_at))
        
        return train_pred_at
    
    

In [372]:
attention_predict(ad_matrix,train_preds,train_y)

1.4480942172734381


,Y_01,Y_02,Y_03,Y_04,Y_05,Y_06,Y_07,Y_08,Y_09,Y_10,Y_11,Y_12,Y_13,Y_14
0,1.866193,1.358355,1.469580,12.240920,30.640781,16.884435,3.837584,-25.365817,-25.411111,-22.259800,24.294982,-25.383214,-25.374317,-25.294914
1,1.549735,1.216865,1.228311,16.737481,32.571930,17.025715,3.248602,-25.971029,-25.936352,-22.083984,24.830434,-25.817488,-25.905789,-25.893818
2,1.401809,0.959306,0.933379,14.752681,31.511012,16.981993,3.030250,-25.795330,-25.815677,-22.006198,24.764622,-25.787140,-25.741022,-25.816050
3,1.474454,1.145797,1.074303,15.904151,33.321804,17.046634,3.098123,-25.640909,-25.448657,-21.846590,24.784018,-25.534141,-25.416638,-25.553164
4,1.276525,0.898161,0.874802,15.269466,32.477584,17.158857,3.129225,-25.408567,-25.356118,-21.434585,25.121273,-25.289368,-25.326830,-25.253269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39602,1.326920,0.983689,1.080132,12.374851,30.768689,16.758388,3.163972,-26.470398,-26.498064,-22.659939,24.385115,-26.408693,-26.445353,-26.439291
39603,1.397816,0.859030,1.083186,11.444981,30.484904,16.624029,3.192100,-26.653382,-26.614476,-23.312937,24.143161,-26.632194,-26.618368,-26.617171
39604,1.250863,0.955167,0.959223,13.040079,29.444337,16.554558,3.209017,-26.548801,-26.591924,-22.848662,24.312292,-26.473429,-26.478979,-26.526944
39605,1.176401,0.679033,0.811531,11.785616,29.362465,16.503588,3.543955,-26.632849,-26.572387,-23.277445,23.919050,-26.515306,-26.562634,-26.533267


In [373]:
test_pred_at_arr=attention_predict(ad_matrix,test_preds)

array([[  1.44907796,   1.28512275,   1.13295338, ..., -25.95233979,
        -25.93646416, -25.95279346],
       [  1.4794086 ,   1.23666465,   1.13516449, ..., -26.11675749,
        -26.11616287, -26.10084166],
       [  1.46429447,   1.18750434,   1.15731511, ..., -25.87311477,
        -25.86051835, -25.87301396],
       ...,
       [  1.22565267,   0.94115352,   0.96523152, ..., -26.52706645,
        -26.49861202, -26.53411282],
       [  1.18826782,   0.8230155 ,   0.9069026 , ..., -26.40697548,
        -26.42156157, -26.45094554],
       [  1.30230593,   0.96239519,   1.00386068, ..., -26.53126239,
        -26.53042642, -26.53753695]])

In [272]:
submit = pd.read_csv('sample_submission.csv')

In [273]:
for idx, col in enumerate(submit.columns):
    if col=='ID':
        continue
    submit[col] = test_pred_at_arr[:,idx-1]

In [276]:
submit.to_csv('lisk_sm_newjeans.csv', index=False)